In [15]:
import os
import nltk
import nltk.corpus
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.naive_bayes import GaussianNB
import string
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from nltk.util import bigrams, trigrams, ngrams
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction import FeatureHasher

In [2]:
df = pd.read_csv ("train.csv")

In [3]:
def clean_text(df):
    
    all_comments = list()
    lines = df["comment"].values.tolist()
    for text in lines:
        text = text.lower()
        
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub("", text)
        
        emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        text = emoji.sub(r'', text)
        
        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"that's", "that is", text)        
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"where's", "where is", text) 
        text = re.sub(r"\'ll", " will", text)  
        text = re.sub(r"\'ve", " have", text)  
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"don't", "do not", text)
        text = re.sub(r"did't", "did not", text)
        text = re.sub(r"can't", "can not", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"have't", "have not", text)
        text = re.sub(r"nt", "not", text)
        
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        
        tokens = word_tokenize(text)
        
        table = str.maketrans('', '', string.punctuation)
        
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        
        words = [w for w in words if not w in stop_words]
        words = ' '.join(words)
        
        all_comments.append(words)
    return all_comments

all_comments = clean_text(df)
all_comments[0:4]

['think prestige poinots not expire ever skins buy available set duration exemple year release another skin vault old one making also limitededition skin also please love god not rerelease skins need grind prestige shop would suck everyone grinded',
 'whats going happen refused asilum appeal',
 'anecdotal evidence anecdotal clearly everyone meanot like people not',
 'look dude due respect music people looks like carti either caught much flak maybe senot polite post inviting discussion instead capitalizing every impactful word post carti']

In [4]:
vect = CountVectorizer()
vect.fit(all_comments)

CountVectorizer()

In [5]:
print("Vocabulary size: {}".format(len(vect.vocabulary_)))

Vocabulary size: 57535


In [6]:
print("Vocabulary content:\n {}".format((vect.vocabulary_)))

Vocabulary content:
 {'think': 50785, 'prestige': 38985, 'poinots': 38136, 'not': 34370, 'expire': 16482, 'ever': 16121, 'skins': 46315, 'buy': 6718, 'available': 3278, 'set': 45087, 'duration': 14444, 'exemple': 16318, 'year': 56894, 'release': 41577, 'another': 1899, 'skin': 46304, 'vault': 54239, 'old': 35088, 'one': 35203, 'making': 30019, 'also': 1378, 'limitededition': 28867, 'please': 37959, 'love': 29455, 'god': 20074, 'rerelease': 41964, 'need': 33496, 'grind': 20676, 'shop': 45655, 'would': 56482, 'suck': 48960, 'everyone': 16143, 'grinded': 20678, 'whats': 55756, 'going': 20127, 'happen': 21976, 'refused': 41366, 'asilum': 2714, 'appeal': 2199, 'anecdotal': 1720, 'evidence': 16159, 'clearly': 8652, 'meanot': 30885, 'like': 28820, 'people': 37036, 'look': 29308, 'dude': 14317, 'due': 14328, 'respect': 42092, 'music': 32942, 'looks': 29314, 'carti': 7314, 'either': 14956, 'caught': 7503, 'much': 32728, 'flak': 17772, 'maybe': 30693, 'senot': 44933, 'polite': 38215, 'post': 384

In [7]:
bag_of_words = vect.transform(all_comments)

In [8]:
bag_of_words.shape

(60000, 57535)

In [9]:
# print("bag_of_words as an array:\n {}".format(bag_of_words.toarray()))
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df['subreddit'], test_size = 0.2, random_state = 0)

In [18]:
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

# TEST

In [10]:
# df_test = pd.read_csv ("test.csv")

In [11]:
# df_test.head()

In [12]:
# cleaned_test = clean_text(df_test)

In [13]:
# print("Each community contains: {}".format(np.bincount(df_test["comment"])))